In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local[1]') \
                    .appName('test') \
                    .getOrCreate()

24/01/12 12:52:11 WARN Utils: Your hostname, codespaces-d00206 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/01/12 12:52:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/12 12:52:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# How to Run Spark SQL Queries

Spark provides several interfaces to execute SQL queries.

## Spark SQL CLI

The Spark SQL CLI is a convenient tool with which you can make basic Spark SQL queries in local mode from the command line. Note that the Spark SQL CLI cannot communicate with the Thrift JDBC server. To start the Spark SQL CLI, run the following in the Spark directory:

```
./bin/spark-sql
```

You configure Hive by placing your hive-site.xml, core-site.xml, and hdfs-site.xml files in conf/. For a complete list of all available options, you can run ```./bin/spark-sql --help```.

## Spark’s Programmatic SQL Interface

In addition to setting up a server, you can also execute SQL in an ad hoc manner via any of Spark’s language APIs. You can do this via the method sql on the SparkSession object. This returns a DataFrame. For example, in Python or Scala, we can run the following:

In [3]:
spark.sql("SELECT 1 + 1").show()

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+



You can express multiline queries quite simply by passing a multiline string into the function. For example, you could execute something like the following code in Python or Scala:

In [ ]:
spark.sql("""SELECT user_id, department, first_name FROM professors
             WHERE department IN
             (SELECT name FROM department WHERE created_date >= '2016-01-01')""")

Even more powerful, you can completely interoperate between SQL and DataFrames, as you see fit. For instance, you can create a DataFrame, manipulate it with SQL, and then manipulate it again as a DataFrame

In [6]:
spark.read.json("../data/flight-data/json/2015-summary.json").createOrReplaceTempView("some_sql_view") # DF => SQL

In [7]:
spark.sql("""
  SELECT DEST_COUNTRY_NAME, sum(count)
  FROM some_sql_view GROUP BY DEST_COUNTRY_NAME
  """)\
    .where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10")\
    .count() # SQL => DF

12

# Catalog

The highest level abstraction in Spark SQL is the Catalog. The Catalog is an abstraction for the storage of metadata about the data stored in your tables as well as other helpful things like databases, tables, functions, and views. The catalog is available in the org.apache.spark.sql.catalog.Catalog package and contains a number of helpful functions for doing things like listing tables, databases, and functions.

# Tables

To do anything useful with Spark SQL, you first need to define tables. Tables are logically equivalent to a DataFrame in that they are a structure of data against which you run commands. The core difference between tables and DataFrames is this: you define DataFrames in the scope of a programming language, whereas you define tables within a database. This means that when you create a table (assuming you never changed the database), it will belong to the default database.


An important thing to note is that in Spark 2.X, tables always contain data. There is no notion of a temporary table, only a view, which does not contain data. This is important because if you go to drop a table, you can risk losing the data when doing so.

## Spark-Managed Tables

Tables store two important pieces of information. The data within the tables as well as the data about the tables; that is, the metadata. You can have Spark manage the metadata for a set of files as well as for the
data. When you define a table from files on disk, you are defining an unmanaged table. When you use saveAsTable on a DataFrame, you are creating a managed table for which Spark will track of all of the relevant information.


This will read your table and write it out to a new location in Spark format. You can see this reflected in the new explain plan. In the explain plan, you will also notice that this writes to the default Hive warehouse location. You can set this by setting the spark.sql.warehouse.dir configuration to the directory of your choosing when you create your SparkSession. By default Spark sets this to /user/hive/warehouse:


You can also see tables in a specific database by using the query show tables IN databaseName, where databaseName represents the name of the database that you want to query.
If you are running on a new cluster or local mode, this should return zero results.

## Creating Tables

Spark is capable of reusing the entire Data Source API within SQL. This means that you do not need to define a table and then load data into it; Spark lets you create one on the fly. For example, here’s a simple way to read in the flight data we worked with in previous chapters:

```sql
CREATE TABLE flights (
    DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
  USING JSON OPTIONS (path '/data/flight-data/json/2015-summary.json')
```

The specification of the USING syntax in the previous example is of significant importance. If you do not specify the format, Spark will default to a Hive SerDe configuration. This has performance implications for future readers and writers because Hive SerDes are much slower than Spark’s native serialization. Hive users can also use the STORED AS syntax to specify that this should be a Hive table.

It is possible to create a table from a query as well. In addition, you can specify to create a table only if it does not currently exist

```sql
CREATE TABLE IF NOT EXISTS flights_from_select
     AS SELECT * FROM flights
```

Finally, you can control the layout of the data by writing out a partitioned dataset

```sql
CREATE TABLE partitioned_flights USING parquet PARTITIONED BY (DEST_COUNTRY_NAME)
  AS SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 5
```

## Creating External Tables

For example, in the example that follows, we create an unmanaged table. Spark will manage the table’s metadata; however, the files are not managed by Spark at all. You create this table by using the CREATE EXTERNAL TABLE statement.

You can view any files that have already been defined by running the following command:
```sql
CREATE EXTERNAL TABLE hive_flights (
    DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '/data/flight-data-hive/'
```

You can also create an external table from a select clause:
```sql
CREATE EXTERNAL TABLE hive_flights_2
  ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
  LOCATION '/data/flight-data-hive/' AS SELECT * FROM flights
```

## Inserting into Tables

```sql
INSERT INTO partitioned_flights
PARTITION (DEST_COUNTRY_NAME="UNITED STATES")
SELECT count, ORIGIN_COUNTRY_NAME FROM flights
WHERE DEST_COUNTRY_NAME='UNITED STATES' LIMIT 12
```
You can optionally provide a partition specification if you want to write only into a certain partition. Note that a write will respect a partitioning scheme, as well; however, it will add additional files only into the end partitions

## Describing Table Metadata

```sql
DESCRIBE TABLE flights_csv

SHOW PARTITIONS partitioned_flights
```

## Refreshing Table Metadata

Maintaining table metadata is an important task to ensure that you’re reading from the most recent set of data. There are two commands to refresh table metadata. REFRESH TABLE refreshes
all cached entries (essentially, files) associated with the table. If the table were previously cached, it would be cached lazily the next time it is scanned:
```sql
REFRESH table partitioned_flights
```

Another related command is REPAIR TABLE, which refreshes the partitions maintained in the catalog for that given table. This command’s focus is on collecting new partition information— an example might be writing out a new partition manually and the need to repair the table accordingly:
```sql
MSCK REPAIR TABLE partitioned_flights
```

## Dropping Tables

```sql
DROP TABLE [IF EXISTS] flights_csv;
```

If you are dropping an unmanaged table (e.g., hive_flights), no data will be removed but you will no longer be able to refer to this data by the table name.

## Caching Tables

```sql
CACHE TABLE flights

UNCACHE TABLE FLIGHTS
```

# Views

A view specifies a set of transformations on top of an existing table—basically just saved query plans, which can be convenient for organizing or reusing your query logic

## Creating Views

To an end user, views are displayed as tables, except rather than rewriting all of the data to a new location, they simply perform a transformation on the source data at query time.

```sql
CREATE VIEW just_usa_view AS
    SELECT * FROM flights WHERE dest_country_name = 'United States'
```

Like tables, you can create temporary views that are available only during the current session and are not registered to a database:

```sql
  CREATE TEMP VIEW just_usa_view_temp AS
    SELECT * FROM flights WHERE dest_country_name = 'United States'
```

Or, it can be a global temp view. Global temp views are resolved regardless of database and are viewable across the entire Spark application, but they are removed at the end of the session:

```sql
CREATE GLOBAL TEMP VIEW just_usa_global_view_temp AS
    SELECT * FROM flights WHERE dest_country_name = 'United States'
```

You can also specify that you would like to overwite a view if one already exists

```sql
CREATE OR REPLACE TEMP VIEW just_usa_view_temp AS
    SELECT * FROM flights WHERE dest_country_name = 'United States'
```

## Dropping Views

```sql
DROP VIEW IF EXISTS just_usa_view;
```

# Databases

Databases are a tool for organizing tables. As mentioned earlier, if you do not define one, Spark will use the default database.

You can see all databases by using the following command:

```sql
SHOW DATABASES
```

## Creating Databases

```sql
CREATE DATABASE some_db
```

## Setting the Database

You might want to set a database to perform a certain query. To do this, use the USE keyword followed by the database name:


```sql
USE some_db
```

After you set this database, all queries will try to resolve table names to this database.

You can query different databases by using the correct prefix:


```sql
SELECT * FROM default.flights
```

You can see what database you’re currently using by running the following command:

```sql
SELECT current_database()
```

## Dropping Databases

```sql
DROP DATABASE IF EXISTS some_db;
```

# Select Statements

```sql
SELECT [ALL|DISTINCT] named_expression[, named_expression, ...]
      FROM relation[, relation, ...]
      [lateral_view[, lateral_view, ...]]
      [WHERE boolean_expression]
      [aggregation [HAVING boolean_expression]]
      [ORDER BY sort_expressions]
      [CLUSTER BY expressions]
      [DISTRIBUTE BY expressions]
      [SORT BY sort_expressions]
      [WINDOW named_window[, WINDOW named_window, ...]]
      [LIMIT num_rows]

  named_expression:
      : expression [AS alias]

  relation:
      | join_relation
      | (table_name|query|relation) [sample] [AS alias]
      : VALUES (expressions)[, (expressions), ...]
            [AS (column_name[, column_name, ...])]

  expressions:
      : expression[, expression, ...]
      
  sort_expressions:
      : expression [ASC|DESC][, expression [ASC|DESC], ...]
```

## case...when...then Statements

```sql
SELECT
    CASE WHEN DEST_COUNTRY_NAME = 'UNITED STATES' THEN 1
         WHEN DEST_COUNTRY_NAME = 'Egypt' THEN 0
         ELSE -1 END
  FROM partitioned_flights
```

# Advanced Topics

## Complex Types

Complex types are a departure from standard SQL and are an incredibly powerful feature that does not exist in standard SQL. There are three core complex types in Spark SQL: structs, lists, and maps.

### Structs

Structs are more akin to maps. They provide a way of creating or querying nested data in Spark. To create one, you simply need to wrap a set of columns (or expressions) in parentheses:

```sql
CREATE VIEW IF NOT EXISTS nested_data AS
    SELECT (DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME) as country, count FROM flights
```

```sql
SELECT * FROM nested_data
```

```sql
SELECT country.DEST_COUNTRY_NAME, count FROM nested_data
```

```sql
SELECT country.*, count FROM nested_data
```

### Lists

There are several ways to create an array or list of values. You can use the collect_list function,
which creates a list of values. You can also use the function collect_set, which creates an array without duplicate values. These are both aggregation functions and therefore can be specified only in aggregations:

```sql
SELECT
    DEST_COUNTRY_NAME as new_name,
    collect_list(count) as flight_counts,
    collect_set(ORIGIN_COUNTRY_NAME) as origin_set
FROM flights
GROUP BY DEST_COUNTRY_NAME
```

You can, however, also create an array manually within a column, as shown here:

```sql
SELECT DEST_COUNTRY_NAME, ARRAY(1, 2, 3) FROM flights
```

You can also query lists by position by using a Python-like array query syntax:

```sql
SELECT DEST_COUNTRY_NAME as new_name, collect_list(count)[0]
FROM flights GROUP BY DEST_COUNTRY_NAME
```

You can also do things like convert an array back into rows. You do this by using the explode function. To demonstrate, let’s create a new view as our aggregation:
```sql
CREATE OR REPLACE TEMP VIEW flights_agg AS
    SELECT DEST_COUNTRY_NAME, collect_list(count) as collected_counts
    FROM flights GROUP BY DEST_COUNTRY_NAME
```

Now let’s explode the complex type to one row in our result for every value in the array. The DEST_COUNTRY_NAME will duplicate for every value in the array, performing the exact opposite of the original collect and returning us to the original DataFrame:
```sql
SELECT explode(collected_counts), DEST_COUNTRY_NAME FROM flights_agg
```

## Functions

To see a list of functions in Spark SQL
```sql
SHOW FUNCTIONS
```

You can also more specifically indicate whether you would like to see the system functions (i.e.,
those built into Spark) as well as user functions:
```sql
SHOW SYSTEM FUNCTIONS

SHOW USER FUNCTIONS

SHOW FUNCTIONS "s*"
-- Optionally, you can include the LIKE keyword
SHOW FUNCTIONS LIKE "collect*"
```

Even though listing functions is certainly useful, often you might want to know more about specific functions themselves. To do this, use the DESCRIBE keyword, which returns the
documentation for a specific function.

### User-defined functions

You can define functions, just as you did before, writing the function in the language of your choice and then registering it appropriately:

In [6]:
def power3(number):
    Double = number * number * number

from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
    
spark.udf.register("power3", power3, DoubleType())

<function __main__.power3(number)>

```sql
SELECT count, power3(count) FROM flights
```

## Subqueries

Correlated subqueries use some information from the outer scope of the query in order to supplement information in the subquery.

Uncorrelated subqueries include no information from the outer scope. 

Each of these queries can return one (scalar subquery) or more values. Spark also includes support for predicate subqueries, which allow for filtering based on values.

### Uncorrelated predicate subqueries

```sql
SELECT * FROM flights
  WHERE origin_country_name IN (SELECT dest_country_name FROM flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5)
```

This query is uncorrelated because it does not include any information from the outer scope of
the query. It’s a query that you can run on its own.

### Correlated predicate subqueries

```sql

  SELECT * FROM flights f1
  WHERE EXISTS (SELECT 1 FROM flights f2
              WHERE f1.dest_country_name = f2.origin_country_name)
  AND EXISTS (SELECT 1 FROM flights f2
              WHERE f2.dest_country_name = f1.origin_country_name)
```

### Uncorrelated scalar queries

```sql
 SELECT *, (SELECT max(count) FROM flights) AS maximum FROM flights
```